The traffic accident data is the key to this analysis. To make sure it's current, instead of downloading a static .csv file it, along with several other datasets from the City of Nashville's OpenData portal, will be periodically downloaded and updated via APIs.

To avoid too many pull requests, all of the data will be pulled here and the relevant information will be exported for use in a separate notebook for EDA.

** This notebook will be revisited and edited as data needs may change for this analysis.

In [167]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
# Needed to pip install sodapy first
from sodapy import Socrata
from geopy.geocoders import Nominatim

**Data Source:** https://data.nashville.gov/Police/Traffic-Accidents/6v6w-hpcw/about_data

In [168]:
client = Socrata("data.nashville.gov", None)

In [169]:
crashes_export = client.get("6v6w-hpcw", limit=200000)
crashes_raw = pd.DataFrame.from_records(crashes_export)
crashes_raw.head()

,accident_number,date_and_time,number_of_motor_vehicles,number_of_injuries,number_of_fatalities,hit_and_run,reporting_officer,collision_type,collision_type_description,weather,...,rpa,precinct,lat,long,mapped_location,:@computed_region_wvby_4s8j,:@computed_region_3aw5_2wv7,:@computed_region_p6sk_2acq,:@computed_region_gxvr_9jxz,property_damage
0,20240132089,2024-02-25T17:44:00.000,2,1,0,False,4010704,4,ANGLE,21,...,8861,SOUTH,36.0745,-86.6387,"{'type': 'Point', 'coordinates': [-86.6387, 36...",1,1,22,22,NaN
1,20240132064,2024-02-25T18:41:00.000,2,1,0,False,4006691,5,SIDESWIPE - SAME DIRECTION,21,...,87060,SOUTH,36.0564,-86.6761,"{'type': 'Point', 'coordinates': [-86.6761, 36...",1,1,28,15,NaN
2,20240131952,2024-02-25T17:45:00.000,1,3,0,False,4004350,0,NOT COLLISION W/MOTOR VEHICLE-TRANSPORT,21,...,8995,HERMIT,36.1412,-86.6284,"{'type': 'Point', 'coordinates': [-86.6284, 36...",1,2,6,38,NaN
3,20240131887,2024-02-25T17:05:00.000,2,0,0,False,4011313,11,Front to Rear,21,...,8603,MIDTOW,36.0811,-86.7584,"{'type': 'Point', 'coordinates': [-86.7584, 36...",1,1,12,3,NaN
4,20240131809,2024-02-25T16:12:00.000,2,1,0,False,4001081,5,SIDESWIPE - SAME DIRECTION,21,...,4005,CENTRA,36.1533,-86.7722,"{'type': 'Point', 'coordinates': [-86.7722, 36...",1,1,20,11,NaN


In [170]:
crashes_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180608 entries, 0 to 180607
Data columns (total 29 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   accident_number              180608 non-null  object
 1   date_and_time                180608 non-null  object
 2   number_of_motor_vehicles     180607 non-null  object
 3   number_of_injuries           180608 non-null  object
 4   number_of_fatalities         180608 non-null  object
 5   hit_and_run                  180594 non-null  object
 6   reporting_officer            180597 non-null  object
 7   collision_type               180598 non-null  object
 8   collision_type_description   180598 non-null  object
 9   weather                      174549 non-null  object
 10  weather_description          174549 non-null  object
 11  illuaccidemination           180326 non-null  object
 12  illumination_description     180326 non-null  object
 13  harmfulcodes  

Now that the DataFrame has been created, some columns can be removed as they're not relevant to this analysis.

In [171]:
crashes_raw = crashes_raw.drop(crashes_raw.columns[[6, 7, 9, 11, 13, 24, 25, 26, 27]], axis=1)

In [172]:
crashes_raw['date'] = pd.to_datetime(crashes_raw['date_and_time']).dt.date
crashes_raw = crashes_raw.astype({'number_of_motor_vehicles': 'float',
                    'number_of_injuries': 'float',
                    'number_of_fatalities': 'float',
                    'hit_and_run': 'bool',
                    'property_damage': 'bool'})

In [173]:
crashes_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180608 entries, 0 to 180607
Data columns (total 21 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   accident_number             180608 non-null  object 
 1   date_and_time               180608 non-null  object 
 2   number_of_motor_vehicles    180607 non-null  float64
 3   number_of_injuries          180608 non-null  float64
 4   number_of_fatalities        180608 non-null  float64
 5   hit_and_run                 180608 non-null  bool   
 6   collision_type_description  180598 non-null  object 
 7   weather_description         174549 non-null  object 
 8   illumination_description    180326 non-null  object 
 9   harmfuldescriptions         178534 non-null  object 
 10  street_address              180602 non-null  object 
 11  city                        180608 non-null  object 
 12  state                       180607 non-null  object 
 13  zip           

For some initial cleaning, all text fields should be converted to the same case, in this case UPPER

In [174]:
crashes_raw['collision_type_description'] = crashes_raw['collision_type_description'].astype(str).str.upper()
crashes_raw['illumination_description'] = crashes_raw['illumination_description'].astype(str).str.upper()
crashes_raw['harmfuldescriptions'] = crashes_raw['harmfuldescriptions'].astype(str).str.upper()
crashes_raw['street_address'] = crashes_raw['street_address'].astype(str).str.upper()
crashes_raw['city'] = crashes_raw['city'].astype(str).str.upper()
crashes_raw['state'] = crashes_raw['state'].astype(str).str.upper()
crashes_raw['precinct'] = crashes_raw['precinct'].astype(str).str.upper()
crashes_raw['weather_description'] = crashes_raw['weather_description'].astype(str).str.upper()
crashes_raw['property_damage'] = crashes_raw['property_damage'].astype(str).str.upper()

In [175]:
crashes_raw.head()

,accident_number,date_and_time,number_of_motor_vehicles,number_of_injuries,number_of_fatalities,hit_and_run,collision_type_description,weather_description,illumination_description,harmfuldescriptions,...,city,state,zip,rpa,precinct,lat,long,mapped_location,property_damage,date
0,20240132089,2024-02-25T17:44:00.000,2.0,1.0,0.0,False,ANGLE,CLEAR,DAYLIGHT,MOTOR VEHICLE IN TRANSPORT,...,ANTIOCH,TN,37013,8861,SOUTH,36.0745,-86.6387,"{'type': 'Point', 'coordinates': [-86.6387, 36...",TRUE,2024-02-25
1,20240132064,2024-02-25T18:41:00.000,2.0,1.0,0.0,False,SIDESWIPE - SAME DIRECTION,CLEAR,DARK - NOT LIGHTED,MOTOR VEHICLE IN TRANSPORT,...,ANTIOCH,TN,37013,87060,SOUTH,36.0564,-86.6761,"{'type': 'Point', 'coordinates': [-86.6761, 36...",TRUE,2024-02-25
2,20240131952,2024-02-25T17:45:00.000,1.0,3.0,0.0,False,NOT COLLISION W/MOTOR VEHICLE-TRANSPORT,CLEAR,DARK - LIGHTED,DITCH,...,NASHVILLE,TN,37214,8995,HERMIT,36.1412,-86.6284,"{'type': 'Point', 'coordinates': [-86.6284, 36...",TRUE,2024-02-25
3,20240131887,2024-02-25T17:05:00.000,2.0,0.0,0.0,False,FRONT TO REAR,CLEAR,DAYLIGHT,MOTOR VEHICLE IN TRANSPORT,...,NASHVILLE,TN,37211,8603,MIDTOW,36.0811,-86.7584,"{'type': 'Point', 'coordinates': [-86.7584, 36...",TRUE,2024-02-25
4,20240131809,2024-02-25T16:12:00.000,2.0,1.0,0.0,False,SIDESWIPE - SAME DIRECTION,CLEAR,DAYLIGHT,MOTOR VEHICLE IN TRANSPORT,...,NASHVILLE,TN,37210,4005,CENTRA,36.1533,-86.7722,"{'type': 'Point', 'coordinates': [-86.7722, 36...",TRUE,2024-02-25


Last, this analysis is only concerned with crashes in the vicinity of Main Street / Gallatin Avenue / Gallatin Pike in East Nashville & Madison.

In [176]:
crashes = crashes_raw[crashes_raw['street_address'].str.contains('GALLATIN|MAIN')]
crashes.head()

,accident_number,date_and_time,number_of_motor_vehicles,number_of_injuries,number_of_fatalities,hit_and_run,collision_type_description,weather_description,illumination_description,harmfuldescriptions,...,city,state,zip,rpa,precinct,lat,long,mapped_location,property_damage,date
41,20240129721,2024-02-24T16:28:00.000,3.0,0.0,0.0,False,ANGLE,CLEAR,DAYLIGHT,MOTOR VEHICLE IN TRANSPORT,...,NASHVILLE,TN,37206,1999,EAST,36.1754,-86.7588,"{'type': 'Point', 'coordinates': [-86.7588, 36...",TRUE,2024-02-24
53,20240129376,2024-02-24T12:23:00.000,2.0,0.0,0.0,False,FRONT TO REAR,CLEAR,DAYLIGHT,MOTOR VEHICLE IN TRANSPORT,...,MADISON,TN,37115,1731,MADISO,36.3044,-86.6895,"{'type': 'Point', 'coordinates': [-86.6895, 36...",TRUE,2024-02-24
95,20240127027,2024-02-23T12:16:00.000,2.0,0.0,0.0,False,FRONT TO REAR,CLEAR,DAYLIGHT,MOTOR VEHICLE IN TRANSPORT,...,MADISON,TN,37115,1731,MADISO,36.3044,-86.6895,"{'type': 'Point', 'coordinates': [-86.6895, 36...",TRUE,2024-02-23
111,20240126477,2024-02-23T07:10:00.000,2.0,0.0,0.0,True,FRONT TO REAR,CLEAR,DAYLIGHT,MOTOR VEHICLE-IN-MOTION OUTSIDE TRAFFICW,...,MADISON,TN,37115,1507,MADISO,36.2511,-86.7182,"{'type': 'Point', 'coordinates': [-86.7182, 36...",TRUE,2024-02-23
123,20240125380,2024-02-22T21:15:00.000,1.0,1.0,0.0,False,NOT COLLISION W/MOTOR VEHICLE-TRANSPORT,RAIN,DARK - LIGHTED,NAN,...,NASHVILLE,TN,37206,1995,EAST,36.1751,-86.7675,"{'type': 'Point', 'coordinates': [-86.7675, 36...",TRUE,2024-02-22


The resulting table is exported to a .csv file for use in the EDA notebook.

In [177]:
crashes.to_csv('../data/clean/crashes.csv')

Now repeat the process for active ROW permits.<br><br>
Source: https://data.nashville.gov/Licenses-Permits/Active-Right-of-Way-Permits/a5tp-4w2v/about_data

In [178]:
row_permits_export = client.get("a5tp-4w2v", limit=2000000)
row_permits_raw = pd.DataFrame.from_records(row_permits_export)
row_permits_raw.head()

,initiated_date,permit,permit_type,permit_description,status,on_street,from_street,to_street,location_address,city,...,company,days_to_work,latitude,longitude,mapped_location,:@computed_region_wvby_4s8j,:@computed_region_3aw5_2wv7,:@computed_region_p6sk_2acq,:@computed_region_gxvr_9jxz,:@computed_region_b9k3_hpc2
0,2023-02-09T11:47:34.000,202305185,DRIVEWAY PERMIT,POURING ENTRANCE AND APRON TO DRIVEWAY,ACTIVE,4108 WESTLAWN DR,WESTLAWN PL,WESTLAWN CT,4108 WESTLAWN DR,NASHVILLE,...,Christy Conway,Monday Tuesday Wednesday Thursday Friday Satur...,36.1393235314126,-86.836318688412,"{'type': 'Point', 'coordinates': [-86.83631868...",1,1,10,46,5
1,2023-06-02T11:01:59.000,202320188,DRIVEWAY PERMIT,RESIDENTIAL DRIVEWAY CONSTRUCTION,ACTIVE,0 KNIGHT DR,MID-BLOCK,JUDY CREEK RD,0 KNIGHT DR,WHITES CREEK,...,Michael L Shular,Monday Tuesday Wednesday Thursday Friday Satur...,36.2626428157731,-86.8253323554335,"{'type': 'Point', 'coordinates': [-86.82533235...",1,2,16,6,1
2,2022-10-06T09:36:29.000,202236666,EXCAVATION PERMIT,STORMWATER INSTALL,ACTIVE,KINGS LN,PHIPPS DR,DRAKES BRANCH RD,KINGS LN,NASHVILLE,...,Middle Tennessee Infrastructure,Tuesday Wednesday Thursday Friday Saturday,36.2175270092533,-86.846531521226,"{'type': 'Point', 'coordinates': [-86.84653152...",1,1,1,34,1
3,2023-04-13T09:09:13.000,202313772,EXCAVATION PERMIT,SEWER/WATER TAP,ACTIVE,4317 CATO RD,CATO RD,GILMORE CROSSING LN,4317 CATO RD,NASHVILLE,...,"R & A PLUMBING, LLC",Monday Tuesday Wednesday Thursday Friday Satur...,36.2138889523239,-86.8666383650662,"{'type': 'Point', 'coordinates': [-86.86663836...",1,1,1,34,1
4,2023-05-15T10:27:04.000,202317822,EXCAVATION PERMIT,PLACING ANCHOR\nJOB# A02G791 (NN)\nSTEP: 1.7,ACTIVE,META DR,WESTCREST DR,ELLENWOOD DR,META DR,NASHVILLE,...,STAR CONSTRUCTION LLC,Tuesday Wednesday Thursday Friday Saturday,36.0594833908917,-86.7349955972504,"{'type': 'Point', 'coordinates': [-86.73499559...",1,1,21,3,3


In [179]:
row_permits_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   initiated_date               23 non-null     object
 1   permit                       23 non-null     object
 2   permit_type                  23 non-null     object
 3   permit_description           23 non-null     object
 4   status                       23 non-null     object
 5   on_street                    23 non-null     object
 6   from_street                  23 non-null     object
 7   to_street                    23 non-null     object
 8   location_address             23 non-null     object
 9   city                         23 non-null     object
 10  zip                          23 non-null     object
 11  scope                        23 non-null     object
 12  scheduled_start              23 non-null     object
 13  scheduled_end                23 non-n

In [180]:
row_permits_raw = row_permits_raw.drop(row_permits_raw.columns[[4, 22, 23, 24, 25, 26]], axis=1)

In [181]:
row_permits_raw['initiated_date'] = pd.to_datetime(row_permits_raw['initiated_date'])
row_permits_raw['scheduled_start'] = pd.to_datetime(row_permits_raw['scheduled_start'])
row_permits_raw['scheduled_end'] = pd.to_datetime(row_permits_raw['scheduled_end'])

In [182]:
row_permits_raw['permit_type'] = row_permits_raw['permit_type'].astype(str).str.upper()
row_permits_raw['permit_description'] = row_permits_raw['permit_description'].astype(str).str.upper()
row_permits_raw['on_street'] = row_permits_raw['on_street'].astype(str).str.upper()
row_permits_raw['from_street'] = row_permits_raw['from_street'].astype(str).str.upper()
row_permits_raw['to_street'] = row_permits_raw['to_street'].astype(str).str.upper()
row_permits_raw['location_address'] = row_permits_raw['location_address'].astype(str).str.upper()
row_permits_raw['city'] = row_permits_raw['city'].astype(str).str.upper()
row_permits_raw['scope'] = row_permits_raw['scope'].astype(str).str.upper()
row_permits_raw['permit_applicant_name'] = row_permits_raw['permit_applicant_name'].astype(str).str.upper()
row_permits_raw['council_district'] = row_permits_raw['council_district'].astype(str).str.upper()
row_permits_raw['conditions_traffic_control'] = row_permits_raw['conditions_traffic_control'].astype(str).str.upper()
row_permits_raw['company'] = row_permits_raw['company'].astype(str).str.upper()

In [183]:
row_permits = row_permits_raw[row_permits_raw['on_street'].str.contains('GALLATIN|MAIN')]
row_permits

,initiated_date,permit,permit_type,permit_description,on_street,from_street,to_street,location_address,city,zip,...,scheduled_start,scheduled_end,permit_applicant_name,council_district,conditions_traffic_control,company,days_to_work,latitude,longitude,mapped_location


In [184]:
row_permits.to_csv('../data/clean/row_permits.csv')

...and Nashville 311 calls<br><br>
Source: https://data.nashville.gov/Public-Services/hubNashville-311-Service-Requests/7qhx-rexh/about_data

In [185]:
nash_311_export = client.get("7qhx-rexh", limit=2000000)
nash_311_raw = pd.DataFrame.from_records(nash_311_export)
nash_311_raw.head()

,case_number,status,case_request,case_subrequest,additional_subrequest,date_time_opened,date_time_closed,case_origin,state_issue,closed_when_created,...,:@computed_region_v67z_xm3t,:@computed_region_kh5x_g7w5,:@computed_region_cfa7_hbpz,:@computed_region_sjpq_96s8,:@computed_region_gisn_y5cm,:@computed_region_b9k3_hpc2,oem_id,contact_type,parent_case,preferred_language
0,966190,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,2022-05-04T17:54:30.000,2022-05-04T17:54:30.000,Phone,False,True,...,2,3,71,1527,4,2,NaN,NaN,NaN,NaN
1,932934,Closed,Public Safety,Abandoned Vehicles,Abandoned Vehicles,2022-04-01T21:01:39.000,2022-04-05T11:29:54.000,Phone,False,False,...,1,9,1018,15550,4,1,NaN,NaN,NaN,NaN
2,932939,Closed,"Trash, Recycling & Litter",Cart Service,Trash - Report a Damaged Cart,2022-04-01T21:04:50.000,2022-04-06T20:14:15.000,Phone,False,False,...,8,33,258,24062,7,8,NaN,NaN,NaN,NaN
3,932935,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,2022-04-01T21:01:47.000,2022-04-01T21:01:47.000,Phone,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,932938,Closed,Public Safety,Request Non-Emergency Police Assistance,NaN,2022-04-01T21:03:10.000,2022-04-01T22:04:31.000,Phone,False,False,...,7,13,50,30406,1,7,NaN,NaN,NaN,NaN


In [186]:
nash_311_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440630 entries, 0 to 1440629
Data columns (total 34 columns):
 #   Column                       Non-Null Count    Dtype 
---  ------                       --------------    ----- 
 0   case_number                  1440630 non-null  object
 1   status                       1440630 non-null  object
 2   case_request                 1440615 non-null  object
 3   case_subrequest              1439291 non-null  object
 4   additional_subrequest        1351181 non-null  object
 5   date_time_opened             1440630 non-null  object
 6   date_time_closed             1426068 non-null  object
 7   case_origin                  1440630 non-null  object
 8   state_issue                  1440630 non-null  object
 9   closed_when_created          1440630 non-null  object
 10  incident_address             980378 non-null   object
 11  incident_city                963768 non-null   object
 12  incident_council_district    965359 non-null   object
 1

In [187]:
nash_311_raw = nash_311_raw.drop(nash_311_raw.columns[[17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]], axis=1)

In [188]:
nash_311_raw['date_time_opened'] = pd.to_datetime(nash_311_raw['date_time_opened'])
nash_311_raw['date_time_closed'] = pd.to_datetime(nash_311_raw['date_time_closed'])

In [189]:
nash_311_raw['status'] = nash_311_raw['status'].astype(str).str.upper()
nash_311_raw['case_request'] = nash_311_raw['case_request'].astype(str).str.upper()
nash_311_raw['case_subrequest'] = nash_311_raw['case_subrequest'].astype(str).str.upper()
nash_311_raw['additional_subrequest'] = nash_311_raw['additional_subrequest'].astype(str).str.upper()
nash_311_raw['case_origin'] = nash_311_raw['case_origin'].astype(str).str.upper()
nash_311_raw['state_issue'] = nash_311_raw['state_issue'].astype(str).str.upper()
nash_311_raw['incident_address'] = nash_311_raw['incident_address'].astype(str).str.upper()
nash_311_raw['incident_city'] = nash_311_raw['incident_city'].astype(str).str.upper()
nash_311_raw['incident_council_district'] = nash_311_raw['incident_council_district'].astype(str).str.upper()
nash_311_raw['contact_type'] = nash_311_raw['contact_type'].astype(str).str.upper()
nash_311_raw['parent_case'] = nash_311_raw['parent_case'].astype(str).str.upper()
nash_311_raw['preferred_language'] = nash_311_raw['preferred_language'].astype(str).str.upper()

In [190]:
nash_311 = nash_311_raw[nash_311_raw['incident_address'].str.contains('GALLATIN|MAIN')]
nash_311

,case_number,status,case_request,case_subrequest,additional_subrequest,date_time_opened,date_time_closed,case_origin,state_issue,closed_when_created,incident_address,incident_city,incident_council_district,incident_zip_code,latitude,longitude,mapped_location,contact_type,parent_case,preferred_language
145,706362,CLOSED,PUBLIC SAFETY,CONTROL NUMBER REQUEST FOR TOWING,CONTROL NUMBER REQUEST FOR TOWING,2021-06-14 03:41:00,2021-06-14 03:41:00,MOBILE APP,FALSE,TRUE,609 GALLATIN PIKE S,MADISON,9,37115,36.2568189,-86.7137494,"{'latitude': '36.2568189', 'longitude': '-86.7...",NAN,NAN,NAN
195,705132,CLOSED,"TRASH, RECYCLING & LITTER",MISSED PICKUP - RECYCLE CART SERVICE,RECYCLE - BACKDOOR,2021-06-11 02:40:00,2021-06-23 15:29:00,HUBNASHVILLE COMMUNITY,FALSE,FALSE,630 MAINSTREAM DR,NASHVILLE,2,37228,36.19805001,-86.81561503,"{'latitude': '36.19805001', 'longitude': '-86....",NAN,NAN,NAN
238,932992,CLOSED,PUBLIC SAFETY,CONTROL NUMBER REQUEST FOR TOWING,CONTROL NUMBER REQUEST FOR TOWING,2022-04-01 22:07:35,2022-04-01 22:07:35,MOBILE APP,FALSE,True,2232 GALLATIN PIKE N,MADISON,10,37115,36.3078416,-86.6849309,"{'latitude': '36.3078416', 'longitude': '-86.6...",NAN,NAN,NAN
263,707700,CLOSED,"STREETS, ROADS & SIDEWALKS",TRAFFIC LIGHT TIMING,TRAFFIC LIGHT TIMING,2021-06-15 14:40:00,2021-06-28 20:16:00,PHONE,FALSE,FALSE,CONFERENCE DR & GALLATIN PIKE N,MADISON,10,37115,36.3027491,-86.6918546,"{'latitude': '36.3027491', 'longitude': '-86.6...",NAN,NAN,NAN
426,932994,CLOSED,PUBLIC SAFETY,CONTROL NUMBER REQUEST FOR TOWING,CONTROL NUMBER REQUEST FOR TOWING,2022-04-01 22:11:51,2022-04-01 22:11:51,MOBILE APP,FALSE,True,2232 GALLATIN PIKE N,MADISON,10,37115,36.3078416,-86.6849309,"{'latitude': '36.3078416', 'longitude': '-86.6...",NAN,NAN,NAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1440051,1476592,CLOSED,PUBLIC SAFETY,CONTROL NUMBER REQUEST FOR TOWING,CONTROL NUMBER REQUEST FOR TOWING,2024-02-05 19:57:35,2024-02-05 19:57:35,MOBILE APP,FALSE,True,901 GALLATIN PIKE S,MADISON,7,37115,36.25324675176617,-86.7167542129755,"{'latitude': '36.25324675176617', 'longitude':...",NAN,NAN,NAN
1440233,1476830,CLOSED,RESOLVED BY HUBNASHVILLE ON FIRST CALL,RESOLVED BY HUBNASHVILLE ON FIRST CALL,RESOLVED BY HUBNASHVILLE ON FIRST CALL,2024-02-05 22:41:08,2024-02-05 22:41:08,PHONE,FALSE,True,601 MAIN ST,NASHVILLE,5,37206,36.1741513,-86.76280469999999,"{'latitude': '36.1741513', 'longitude': '-86.7...",NAN,NAN,NAN
1440263,1476891,CLOSED,PUBLIC SAFETY,ABANDONED VEHICLES,ABANDONED VEHICLES - PUBLIC PROPERTY,2024-02-06 00:08:41,2024-02-13 18:20:47,HUBNASHVILLE COMMUNITY,FALSE,False,535 MAIN ST,NASHVILLE,5,37206,36.17392356656374,-86.76307163835396,"{'latitude': '36.17392356656374', 'longitude':...",NAN,NAN,NAN
1440329,711441,IN PROGRESS,"STREETS, ROADS & SIDEWALKS",REQUEST NEW SIGN,NAN,2021-06-20 02:01:58,NaT,HUBNASHVILLE COMMUNITY,FALSE,False,MAIN STREET,NASHVILLE,19,37219,36.163,-86.782,"{'latitude': '36.163', 'longitude': '-86.782'}",NAN,NAN,NAN


In [191]:
nash_311.to_csv('../data/clean/nash_311.csv')

...and building permits<br><br>
Source: https://data.nashville.gov/Licenses-Permits/Building-Permits-Issued/3h5w-q8b7/about_data

In [192]:
bdlg_permits_export = client.get("3h5w-q8b7", limit=2000000)
bdlg_permits_raw = pd.DataFrame.from_records(bdlg_permits_export)
bdlg_permits_raw.head()

,permit,permit_type_description,permit_subtype_description,parcel,date_entered,date_issued,const_cost,address,city,state,...,:@computed_region_gisn_y5cm,:@computed_region_v3ji_vzam,:@computed_region_c9xn_skx3,:@computed_region_sjpq_96s8,:@computed_region_kh5x_g7w5,:@computed_region_yf9r_ed6g,:@computed_region_fvtq_wnma,:@computed_region_p6sk_2acq,:@computed_region_b9k3_hpc2,:@computed_region_gxvr_9jxz
0,2019040337,Building Commercial - New,"Kennel / Stable, Kennel / Pound",08000007300,2019-07-09T00:00:00.000,2021-02-24T00:00:00.000,110889,1016 CAMILLA CALDWELL LN,NASHVILLE,TN,...,6,2005,1,16168,1,1,23,1,1,34
1,2020014026,Building Commercial - Rehab,"Hospice, Institutional I-2",09211039000,2020-03-04T00:00:00.000,2021-04-01T00:00:00.000,420926,2300 PATTERSON ST,NASHVILLE,TN,...,5,2005,15,30406,15,15,25,25,7,40
2,2020044553,Building Commercial - Rehab,Restaurant (Full Service),07115048800,2020-07-20T00:00:00.000,2021-03-12T00:00:00.000,14400,1315 DICKERSON PIKE,NASHVILLE,TN,...,2,2005,6,15550,6,6,31,13,7,16
3,2020046770,Building Residential - New,"Accessory Structure, Shed / Storage Bldg",05011010700,2020-07-28T00:00:00.000,2022-09-07T00:00:00.000,15070,3212 RAINWOOD DR,NASHVILLE,TN,...,4,2005,9,15550,9,9,31,16,1,16
4,2020046752,Building Residential - Addition,Single Family Residence,01500012800,2020-07-28T00:00:00.000,2021-03-30T00:00:00.000,95780,7495 WILKINSON RD,JOELTON,TN,...,6,2005,1,28815,1,1,45,1,1,43


In [193]:
bdlg_permits_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33774 entries, 0 to 33773
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   permit                       33774 non-null  object
 1   permit_type_description      33774 non-null  object
 2   permit_subtype_description   33774 non-null  object
 3   parcel                       33774 non-null  object
 4   date_entered                 33774 non-null  object
 5   date_issued                  33774 non-null  object
 6   const_cost                   33773 non-null  object
 7   address                      33774 non-null  object
 8   city                         33773 non-null  object
 9   state                        33774 non-null  object
 10  zip                          33773 non-null  object
 11  subdivision_lot              33773 non-null  object
 12  contact                      33774 non-null  object
 13  per_ty                       33

In [194]:
bdlg_permits_raw = bdlg_permits_raw.drop(bdlg_permits_raw.columns[[21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]], axis=1)

In [195]:
bdlg_permits_raw['date_entered'] = pd.to_datetime(bdlg_permits_raw['date_entered'])
bdlg_permits_raw['date_issued'] = pd.to_datetime(bdlg_permits_raw['date_issued'])

In [196]:
bdlg_permits_raw['permit_type_description'] = bdlg_permits_raw['permit_type_description'].astype(str).str.upper()
bdlg_permits_raw['permit_subtype_description'] = bdlg_permits_raw['permit_subtype_description'].astype(str).str.upper()
bdlg_permits_raw['address'] = bdlg_permits_raw['address'].astype(str).str.upper()
bdlg_permits_raw['city'] = bdlg_permits_raw['city'].astype(str).str.upper()
bdlg_permits_raw['state'] = bdlg_permits_raw['state'].astype(str).str.upper()
bdlg_permits_raw['contact'] = bdlg_permits_raw['contact'].astype(str).str.upper()
bdlg_permits_raw['per_ty'] = bdlg_permits_raw['per_ty'].astype(str).str.upper()
bdlg_permits_raw['permit_subtype'] = bdlg_permits_raw['permit_subtype'].astype(str).str.upper()
bdlg_permits_raw['purpose'] = bdlg_permits_raw['purpose'].astype(str).str.upper()
bdlg_permits_raw['council_dist'] = bdlg_permits_raw['council_dist'].astype(str).str.upper()
bdlg_permits_raw['census_tract'] = bdlg_permits_raw['census_tract'].astype(str).str.upper()
bdlg_permits_raw['case_status'] = bdlg_permits_raw['case_status'].astype(str).str.upper()

In [212]:
bdlg_permits = bdlg_permits_raw[bdlg_permits_raw['address'].str.contains('GALLATIN|MAIN')].reset_index()
bdlg_permits

,index,permit,permit_type_description,permit_subtype_description,parcel,date_entered,date_issued,const_cost,address,city,...,subdivision_lot,contact,per_ty,permit_subtype,ivr_trk,purpose,council_dist,census_tract,case_status,mapped_location
0,25,2020075628,BUILDING COMMERCIAL REHAB STORM DAMAGE,"GENERAL OFFICE, PROFESSIONAL SERVICES",08212026600,2020-12-04,2021-03-03,450000,921 MAIN ST,NASHVILLE,...,P/O LOTS 2 & 3 PHILLIPS SILVERDENE,COMMERCIAL INDUSTRIAL CONSTRUCTION INC,CACL,CAD02B016,3902108,TORNADO DAMAGE REBUILD FUTURE TENANT TO OBTAIN...,5,37011900,DONE,"{'latitude': '36.177084016', 'longitude': '-86..."
1,45,2021006138,BUILDING SIGN PERMIT,SIGN - GROUND / WALL SIGNS,07213038500,2021-01-29,2021-02-26,3560,2500 GALLATIN AVE,NASHVILLE,...,LOT 1 CHESTER AVE SUB,TRI STAR TRANSPORT LLC,CASN,CAZ02A001,3920404,TO ERECT 90SF FACE REPLACEMENT ON EXISTING PYL...,6,37011700,EXPIRED,"{'latitude': '36.195368006', 'longitude': '-86..."
2,72,2021011848,BUILDING COMMERCIAL - NEW,"MULTIFAMILY, APT / TWNHOME > 5 UNIT BLDG",08212031300,2021-02-27,2021-09-09,2025671,918 MAIN ST,NASHVILLE,...,PT LOTS 5 & 6 R W BROWN ADDN,"POWELL ARCHITECTURE & BUILDING STUDIO, LLC",CACN,CAA03R399,3933909,"MUG-A, UDO, MDHA, UZO TO CONSTRUCT MIXED USE ...",6,37019200,DONE,"{'latitude': '36.176308019', 'longitude': '-86..."
3,117,2021019499,BUILDING SIGN PERMIT,SIGN - GROUND / WALL SIGNS,08309000500,2021-04-01,2021-10-04,6000,969 MAIN ST,NASHVILLE,...,"LOTS 50-53, 55, & 56, PT LOTS 54 & 57 PHILLIPS...",TRI STAR TRANSPORT LLC,CASN,CAZ02A001,3959010,REHAB PERMIT 2021026667 FOR NEW TENANT SNOOZE ...,5,37011900,DONE,"{'latitude': '36.178055981', 'longitude': '-86..."
4,164,2021024771,BUILDING COMMERCIAL REHAB STORM DAMAGE,"RETAIL, DEPARTMENT / RETAIL STORES",08216001700,2021-04-21,2021-08-03,50000,715 MAIN ST,NASHVILLE,...,P/O LOTS 8 & 9 W. L. FOSTER 1ST ADDN.,"CRAFTLOGIC CONSTRUCTION, INC.",CACL,CAF20M001,3970890,***MARCH 3 2020 TORNADO DAMAGE*** TO REPLAC...,5,37011900,EXPIRED,"{'latitude': '36.175373018', 'longitude': '-86..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,33387,2023041244,BUILDING SIGN PERMIT,SIGN - GROUND / WALL SIGNS,02600015800,2023-06-22,2023-07-06,4200,2350 GALLATIN PIKE N,MADISON,...,LOT 2 DISKIN/BANNEN SUBDIVISION,GUY DEARMON,CASN,CAZ02A001,4340079,TO ERECT 43SF FACE CHANGE ON EXISTING MONUMENT...,10,37010401,ISSUED,"{'latitude': '36.310600986', 'longitude': '-86..."
461,33460,2023067185,BUILDING COMMERCIAL - REHAB,"BAR OR NIGHTCLUB, TAVERN AND BARS",08305012900,2023-09-13,2023-11-17,300000,604 GALLATIN AVE 202,NASHVILLE,...,"LOT 1 EDGEFIELD HOSPITAL, INC.",SMITH BUILDING GROUP LLC,CACR,CAF06A204,4380932,TO CONDUCT GENERAL RENOVATIONS FOR NEW TENANT;...,5,37011900,ISSUED,"{'latitude': '36.182935003', 'longitude': '-86..."
462,33464,2023095629,BUILDING SIGN PERMIT,SIGN - GROUND / WALL SIGNS,07213009100,2023-12-04,2023-12-12,650,2521 GALLATIN AVE,NASHVILLE,...,LOT 2 TREVECCA SUB RENRAW,CONCORDIA DEVELOPMENT INC,CASN,CAZ02A001,4422898,TO ERECT 2 ILLUMINATED WALL SIGNS FOR CHICAGO’...,5,37011700,ISSUED,"{'latitude': '36.195780982', 'longitude': '-86..."
463,33488,2023060921,BUILDING SIGN PERMIT,SIGN - GROUND / WALL SIGNS,07210001700,2023-08-25,2023-09-29,1200,2900 GALLATIN PIKE,NASHVILLE,...,LOTS 1&2 BRANSFORD REALTY SUBD.,AAA SIGN ME UP LLC,CASN,CAZ02A001,4371125,TO REFACE AN EXISTING POLE SIGN FOR VAPETOWN. ...,6,37011400,ISSUED,"{'latitude': '36.202570994', 'longitude': '-86..."


Need the coordinate data from the `mapped location` column

In [213]:
bdlg_permits['lat'] = [bdlg_permits['mapped_location'][x].get('latitude') for x in range(0, len(bdlg_permits))]
bdlg_permits['long'] = [bdlg_permits['mapped_location'][x].get('longitude') for x in range(0, len(bdlg_permits))]

In [214]:
bdlg_permits.to_csv('../data/clean/bldg_permits.csv')

...and Short Term Rental Permits<br><br>
Source: https://data.nashville.gov/Licenses-Permits/Residential-Short-Term-Rental-Permits/2z82-v8pm/about_data

In [199]:
str_export = client.get("2z82-v8pm", limit=100000)
strs_raw = pd.DataFrame.from_records(str_export)
strs_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15179 entries, 0 to 15178
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   permit                       15179 non-null  object
 1   applicant                    15164 non-null  object
 2   contact                      14783 non-null  object
 3   permit_subtype_description   15179 non-null  object
 4   permit_status                15179 non-null  object
 5   parcel                       15179 non-null  object
 6   date_entered                 15179 non-null  object
 7   date_issued                  13879 non-null  object
 8   expiration_date              15175 non-null  object
 9   address                      15179 non-null  object
 10  city                         15178 non-null  object
 11  state                        15178 non-null  object
 12  zip                          15179 non-null  object
 13  subdivision_lot              15

In [200]:
strs_raw = strs_raw.drop(strs_raw.columns[[25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]], axis=1)

In [201]:
strs_raw['date_entered'] = pd.to_datetime(strs_raw['date_entered'])
strs_raw['date_issued'] = pd.to_datetime(strs_raw['date_issued'])
strs_raw['expiration_date'] = pd.to_datetime(strs_raw['expiration_date'])

In [202]:
strs_raw['applicant'] = strs_raw['applicant'].astype(str).str.upper()
strs_raw['contact'] = strs_raw['contact'].astype(str).str.upper()
strs_raw['permit_subtype_description'] = strs_raw['permit_subtype_description'].astype(str).str.upper()
strs_raw['permit_status'] = strs_raw['permit_status'].astype(str).str.upper()
strs_raw['address'] = strs_raw['address'].astype(str).str.upper()
strs_raw['city'] = strs_raw['city'].astype(str).str.upper()
strs_raw['state'] = strs_raw['state'].astype(str).str.upper()
strs_raw['permit_subtype'] = strs_raw['permit_subtype'].astype(str).str.upper()
strs_raw['purpose'] = strs_raw['purpose'].astype(str).str.upper()
strs_raw['permit_owner_name'] = strs_raw['permit_owner_name'].astype(str).str.upper()
strs_raw['permit_owner_addr2'] = strs_raw['permit_owner_addr2'].astype(str).str.upper()
strs_raw['permit_owner_city'] = strs_raw['permit_owner_city'].astype(str).str.upper()
strs_raw['permit_owner_state'] = strs_raw['permit_owner_state'].astype(str).str.upper()
strs_raw['permit_type'] = strs_raw['permit_type'].astype(str).str.upper()
strs_raw['permit_subtype'] = strs_raw['permit_subtype'].astype(str).str.upper()
strs_raw['council_dist'] = strs_raw['council_dist'].astype(str).str.upper()
strs_raw['census_tract'] = strs_raw['census_tract'].astype(str).str.upper()
strs_raw['permit_owner_addr1'] = strs_raw['permit_owner_addr1'].astype(str).str.upper()
strs_raw['permit_owner_addr3'] = strs_raw['permit_owner_addr3'].astype(str).str.upper()

In [203]:
rentals = strs_raw[strs_raw['zip'].isin(['37206', '37216', '37115'])].reset_index()
rentals

,index,permit,applicant,contact,permit_subtype_description,permit_status,parcel,date_entered,date_issued,expiration_date,...,permit_owner_city,permit_owner_state,permit_owner_zip,permit_type,permit_subtype,council_dist,census_tract,mapped_location,permit_owner_addr1,permit_owner_addr3
0,1,T2017008702,"TREHY, KEITH","TREHY, KEITH",SHORT TERM RENTAL - NON-OWNER OCCUPIED,EXPIRED,08304007300,2017-02-06,NaT,2017-05-08,...,NASHVILLE,TN,37206,CASR,CAZ10A002,7,37011600,"{'latitude': '36.189219', 'longitude': '-86.71...",NAN,NAN
1,2,2017055846,"HALE, JEFFREY L.",JEFFREY HALE,SHORT TERM RENTAL - OWNER OCCUPIED,ISSUED,08212026000,2017-08-25,2017-09-01,2024-09-01,...,HENDERSONVILLE,TN,37075,CASR,CAZ10A001,5,37011900,"{'latitude': '36.178000996', 'longitude': '-86...",NAN,NAN
2,16,2023047359,"OWNBEY, JASON","OWNBEY, JASON",SHORT TERM RENTAL - OWNER OCCUPIED,ISSUED,09402023100,2023-07-19,2023-07-19,2024-07-19,...,NASHVILLE,TN,37206,CASR,CAZ10A001,6,37012200,"{'latitude': '36.166715015', 'longitude': '-86...",NAN,NAN
3,17,2022030944,"BURKART, JAMES","BURKART, JAMES",SHORT TERM RENTAL - OWNER OCCUPIED,ISSUED,06114015300,2022-05-10,2022-05-10,2024-05-10,...,NASHVILLE,TN,37216,CASR,CAZ10A001,7,37011200,"{'latitude': '36.214595002', 'longitude': '-86...",NAN,NAN
4,21,2019021499,SARA CLINE,SARA CLINE,SHORT TERM RENTAL - OWNER OCCUPIED,EXPIRED,08208037200,2019-04-12,2019-04-19,2020-04-19,...,NASHVILLE,TN,37206,CASR,CAZ10A001,5,37011700,"{'latitude': '36.185369988', 'longitude': '-86...",NAN,NAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2867,15124,2021051008,"11TH AVENUE PARTNERS, LLC",DWELL NASHVILLE,SHORT TERM RENTAL � MULTIFAMILY,ISSUED,082121D00200CO,2021-08-10,2021-08-13,2024-08-13,...,NASHVILLE,TN,37220,CASR,CAZ10A003,5,37011900,"{'latitude': '36.179004016', 'longitude': '-86...",BRAD THOMASON,NAN
2868,15126,2020002749,"STRATON, ANDREW & MARIKO","STRATON, ANDREW & MARIKO",SHORT TERM RENTAL - OWNER OCCUPIED,CANCELLED,08305036900,2020-01-14,2020-02-26,2023-02-26,...,NASHVILLE,TN,37206,CASR,CAZ10A001,6,37012100,"{'latitude': '36.184620017', 'longitude': '-86...",NAN,NAN
2869,15129,T2017030096,"NELSON, CANDICE & MATTHEW",CANDICE NELSON,SHORT TERM RENTAL - OWNER OCCUPIED,CANCELLED,08314008300,2017-05-05,NaT,2017-08-03,...,NASHVILLE,TN,37206,CASR,CAZ10A001,6,37012200,"{'latitude': '36.174626987', 'longitude': '-86...",NAN,NAN
2870,15140,2023037519,"MOORE, JONATHAN E. & DONNA KIMBERLY & ET AL","MOORE, JONATHAN E. & DONNA KIMBERLY & ET AL",SHORT TERM RENTAL - OWNER OCCUPIED,ISSUED,05207008600,2023-06-07,2023-06-07,2024-06-07,...,MADISON,TN,37115,CASR,CAZ10A001,9,37010601,"{'latitude': '36.247076009', 'longitude': '-86...",NAN,NAN


Need the coordinate data from the `mapped location` column

In [204]:
rentals['lat'] = [rentals['mapped_location'][x].get('latitude') for x in range(0, len(rentals))]
rentals['long'] = [rentals['mapped_location'][x].get('longitude') for x in range(0, len(rentals))]

In [205]:
rentals.to_csv('../data/clean/rentals.csv')

...and the pedestrian signal inventory<br><br>
Source: https://data.nashville.gov/Transportation/Pedestrian-Signal-Inventory-and-ADA-Self-Assessmen/6xet-f7u7/about_data

In [206]:
ped_sig = client.get("6xet-f7u7", limit=1000000)
ped_inv = pd.DataFrame.from_records(ped_sig)
ped_inv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3153 entries, 0 to 3152
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   intersectionid               3153 non-null   object
 1   onst                         3153 non-null   object
 2   crst                         3153 non-null   object
 3   quad                         3151 non-null   object
 4   inspector                    3153 non-null   object
 5   date                         3153 non-null   object
 6   evnt_lat                     3136 non-null   object
 7   evnt_lon                     3136 non-null   object
 8   evnt_map                     3153 non-null   object
 9   evnt_type                    3150 non-null   object
 10  ped_signal_pres              3150 non-null   object
 11  oa_cmnt                      3150 non-null   object
 12  mapped_location              3136 non-null   object
 13  :@computed_region_wvby_4s8j  3136

In [207]:
ped_inv = ped_inv.drop(ped_inv.columns[[13, 14, 15, 16]], axis=1)

In [208]:
ped_inv['onst'] = ped_inv['onst'].astype(str).str.upper()
ped_inv['crst'] = ped_inv['crst'].astype(str).str.upper()
ped_inv['quad'] = ped_inv['quad'].astype(str).str.upper()
ped_inv['inspector'] = ped_inv['inspector'].astype(str).str.upper()
ped_inv['evnt_type'] = ped_inv['evnt_type'].astype(str).str.upper()

In [209]:
ped_inv = ped_inv[(ped_inv['onst'].str.contains('GALLATIN|MAIN')) | (ped_inv['crst'].str.contains('GALLATIN|MAIN'))]

In [210]:
ped_inv['date'] = pd.to_datetime(ped_inv['date'])

In [211]:
ped_inv.to_csv('../data/clean/ped_inv.csv')